# Semantic Search using ELSER text expansion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joemcelroy/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/03-ELSER.ipynb)


Learn how to use the [Elastic Learned Sparse Encoder](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) for text expansion-powered semantic search.

# 🧰 Requirements

For this example, you will need:

- Python 3.6 or later
- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?elektra=en-ess-sign-up-page))
- The [ELSER](https://www.elastic.co/guide/en/machine-learning/8.8/ml-nlp-elser.html) model installed on your Elastic deployment
- The [Elastic Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html)


# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Setup ELSER
To use ELSER, you must have the [appropriate subscription]() level
for semantic search or the trial period activated.

Follow these [instructions](https://www.elastic.co/guide/en/machine-learning/8.8/ml-nlp-elser.html#trained-model) to download and deploy ELSER in the Kibana UI or using the Dev Tools **Console**.

(Console commands in comments 👇)
<!-- # download elser model

```json
PUT _ml/trained_models/.elser_model_1
{
  "input": {
	"field_names": ["text_field"]
  }
}
``` -->
<!-- # deploy model
```json
POST _ml/trained_models/.elser_model_1/deployment/_start?deployment_id=for_search -->

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`


In [ ]:
!pip install elasticsearch

Next, we need to import the modules we need.
🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpassimport json

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [ ]:
print(client.info())

Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

To use the ELSER model at index time, we'll need to create an index mapping that supports a [`text_expansion`](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-text-expansion-query.html) query.
The mapping must include a field of type [`rank_features`](https://www.elastic.co/guide/en/elasticsearch/reference/current/rank-features.html) to work with our feature vectors of interest.
This field contains the token-weight pairs the ELSER model created based on the input text.

Let's create an index named `elser-movies` with the mappings we need.


In [ ]:
INDEX = 'elser-movies'
client.indices.create(
            index=INDEX,
            settings={
                "index": {
                    "number_of_shards": 1,
                    "number_of_replicas": 1
                }
            },
            mappings={
    "properties": {
      "genre": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "keyScene": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "plot": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "released": {
        "type": "integer"
      },
      "runtime": {
        "type": "integer"
      },
      "title": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "ml.tokens": {
        "type": "rank_features"
      },
      "keyScene": {
        "type": "text"
      }
  }
}
)

# Create an ingest pipeline with an inference processor to use ELSER

In order to use ELSER on our Elastic Cloud deployment we'll need to create an ingest pipeline that contains an inference processor that runs the ELSER model.
Let's add that pipeline using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method.

In [ ]:

client.ingest.put_pipeline(id="elser-v1-test", body={
    "processors": [
    {
      "inference": {
        "model_id": ".elser_model_1",
        "target_field": "ml",
        "field_map": {
          "keyScene": "text_field",
          "plot": "text_field"
        },
        "inference_config": {
          "text_expansion": {
            "results_field": "tokens"
          }
        }
      }
    }
  ]
})

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the machine learning model to be used. In this example, the model ID is set to `.elser_model_1`.
- `target_field`: Defines the field where the inference result will be stored. Here, it is set to `ml`.
- `text_expansion`: Configures text expansion options for the inference process.
In this example, the inference results will be stored in a field named "tokens".

# Create index and mapping for test data


We have some test data in a `json` file at this [URL](https://raw.githubusercontent.com/leemthompo/notebook-tests/main/12-movies.json).
Let's load that into our Elastic deployment.
First we'll create an index named `search-movies` to store that data.

In [ ]:
client.indices.create(
    index="search-movies",
    mappings= {
    "properties": {
      "genre": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "keyScene": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "plot": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "released": {
        "type": "integer"
      },
      "runtime": {
        "type": "integer"
      },
      "title": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      }
    }
})

# Upload sample data

> ⚠ To use the UI to upload data, follow the approach described [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/semantic-search-elser.html#load-data).

Let's upload the JSON data.
The dataset provides information on twelve iconic films.
Each film's entry includes its title, runtime, plot summary, a key scene, genre classification, and release year.

In [ ]:
url = "https://raw.githubusercontent.com/leemthompo/notebook-tests/main/12-movies.json"

# Send a request to the URL and get the response
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

def create_index_body(doc):
    """ Generate the body for an Elasticsearch document. """
    return {
        "_index": "search-movies",
        "_source": doc,
    }

# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `search-movies` index!")




# Ingest the data through the inference ingest pipeline

Create tokens from the text by reindexing the data throught the inference pipeline that uses ELSER as the inference model.

In [ ]:
client.reindex(wait_for_completion=False,
               source={
                  "index": "search-movies"
    },
               dest= {
                  "index": "elser-movies",
                  "pipeline": "elser-v1-test"
    }
)

# Confirm documents are indexed with additional fields

A successful API call in the previous step returns a task ID to monitor the job's progress.
Use the [task management API](https://www.elastic.co/guide/en/elasticsearch/reference/current/tasks.html) to check progress.
You can also monitor this task using the **Trained Models** UI in Kibana, selecting the **Pipelines** tab under **ELSER**.

Call the following, replacing `<task_id>` with the task id returned in the previous step.

In [ ]:
client.tasks.get(task_id='cxy4bU9ASFKpFgZUpa-jnA:19545263')

Inspect a new document to confirm that it now has an `"ml": {"tokens":...}` field that contains a list of new, additional terms.
These terms are the **text expansion** of the field(s) you targeted for ELSER inference.
ELSER essentially creates a tree of expanded terms to improve the semantic searchability of your documents.
We'll be able to search these documents using a `text_expansion` query.

But first let's start with a simple keyword search, to see how ELSER delivers semantically relevant results out of the box.

# Keyword match

## Successful match

Let's start by assuming a user queries the data set and hits an exact match.
BM25 is perfect for exact keyword matches.
Imagine our user remembers a movie where a child's spinning top was a recurring image.
They search for `spinning top` and because these exact words are used in the key scene description, we get a perfect hit.


In [ ]:
response = client.search(
    index="elser-movies",
    query= {
            "match": {
                "keyScene": "spinning top"
            }
        }
)
for hit in response['hits']['hits']:
    doc_id = hit['_id']
    score = hit['_score']
    title = hit['_source']['title']
    text = hit['_source']['keyScene']
    print(f"\nTitle: {title}\nKey scene description: {text}\n")

## Unsuccessful match

Unfortunately, searches that rely on exact matches are brittle.
What if you can't remember the exact name of the thing you're searching for?
Who knows what a spinning top is anyway?

Imagine I can only think of the word `child toy` to describe this apparatus?
A match query won't find any relevant documents.

In [ ]:
response = client.search(
    index="elser-movies",
    query= {
            "match": {
                "keyScene": "child toy"
            }
        }
)
hits = response['hits']['hits']

if not hits:
    print("No matches found")
else:
    for hit in hits:
        doc_id = hit['_id']
        score = hit['_score']
        title = hit['_source']['title']
        text = hit['_source']['keyScene']
        print(f"\nTitle: {title}\nKey scene description: {text}\n")


So it turns out classical term matching strategies are very good, if you know precisely what you're looking for.
But they break down when a user has a hard time articulating what they're trying to find.
Here's where semantic search shines.
It helps capture a user's intent or meaning better, without relying on brittle term matches.

Traditional dense vector based similarity strategies require you to generate embeddings for your data and then map queries into the same mathematical space as the data.
This works well but is time consuming and requires a lot of legwork.
The beauty of the Elastic Learned Sparse Encoder model is that it works out-of-the-box, without the need to fine tune on your data.

The Elastic Learned Sparse Encoder creates a tree of expanded terms, adds them to your documents, improving their semantic searchability.
The fields that you targeted for inference are now enriched with a range of relevant synonyms and related terms, that increase the probability of a successful search.

# Semantic search with the `text_expansion` query

Let's test out semantic search using the Elastic Learned Sparse Encoder, and see if we can improve our earlier unsuccessful search, using the query `child toy`.

To perform semantic search using the Elastic Learned Sparse Encoder, you need the following:
- A `text_expansion` query
- Query text
   - In this example we use `child toy`
- ELSER model ID

In [ ]:
response = client.search(index='elser-movies', size=3,
              query={
                  "text_expansion": {
                  "ml.tokens": {
                      "model_id":".elser_model_1",
                      "model_text":"child toy"
                      
        }
    }
}
)

for hit in response['hits']['hits']:
    doc_id = hit['_id']
    score = hit['_score']
    title = hit['_source']['title']
    text = hit['_source']['keyScene']
    print(f"Score: {score}\nTitle: {title}\nKey scene description: {text}\n")

Success! Out of the box ELSER has taken a fuzzy, but semantically similar query and found the correct match.
Our user has found the movie they're looking for!